# Tarefa 03 de Modelagem e Simulação

**Alunos:** <br> Vinícius Lima Carvalho (12221BCC018) <br>
        Osvaldo Pagioli de Lollo Silva (12221BCC047)

### Questão 4)b)

In [5]:
import numpy as np

# Matriz de probabilidades
P = np.array([
    [0, 1, 0, 0],      # estado 0 -> 1
    [0, 0, 1, 0],      # estado 1 -> 2
    [0, 0.5, 0, 0.5],  # estado 2 -> 1 ou 3
    [1, 0, 0, 0]       # estado 3 -> 0
])

# Taxas de saída
lambdas = np.array([1.0, 1.0, 1.0, 2.0])  # λ0=λ1=λ2=1, λ3=2

# Parâmetros da simulação
N = 1_000_000   # número de saltos
state = 0       # começa no estado 0
times = np.zeros(4)  # acumular tempos em cada estado
rng = np.random.default_rng(seed=42)

for _ in range(N):
    delta_t = rng.exponential(1.0 / lambdas[state])
    times[state] += delta_t

    state = rng.choice(4, p=P[state])

# Normaliza para obter a distribuição limitante
pi_est = times / times.sum()

print("Estimativa por Monte Carlo:")
for i, p in enumerate(pi_est):
    print(f"pi[{i}] ~ {p:.4f}")

# Comparação com valor teórico
pi_teorico = np.array([2/11, 4/11, 4/11, 1/11])
print("\nDistribuição teórica:", pi_teorico)


Estimativa por Monte Carlo:
pi[0] ~ 0.1816
pi[1] ~ 0.3634
pi[2] ~ 0.3643
pi[3] ~ 0.0907

Distribuição teórica: [0.18181818 0.36363636 0.36363636 0.09090909]


### Questão 5)

In [8]:
import numpy as np

# parâmetros
taxa_falha = 1/100  # taxa de falha por máquina
taxa_reparo = 1/2     # taxa de reparo
n = 4                 # número de máquinas

# construir geradora G
G = np.zeros((n+1, n+1))

for i in range(n+1):
    if i > 0:  # falha de uma das i máquinas
        G[i, i-1] = i * taxa_falha
    if i < n:  # reparo de uma máquina
        G[i, i+1] = taxa_reparo

    G[i, i] = -np.sum(G[i])  # diagonal

print("(a) Matriz geradora G:")
print(G)

# resolver distribuição estacionária: pi G = 0, sum(pi)=1
A = np.copy(G.T)
A[-1] = np.ones(n+1)   # substitui última eq pela normalização
b = np.zeros(n+1)
b[-1] = 1
pi = np.linalg.solve(A, b)


# (b) probabilidade de todas estarem quebradas (estado 0)
print(f"\n(b) Probabilidade de todas estarem quebradas: {pi[0]:.8f}")

# (c) número médio de máquinas operantes
esperado = sum(i * pi[i] for i in range(n+1))
print(f"(c) Número médio de máquinas operantes:{esperado:.4f}")


Matriz geradora G:
[[-0.5   0.5   0.    0.    0.  ]
 [ 0.01 -0.51  0.5   0.    0.  ]
 [ 0.    0.02 -0.52  0.5   0.  ]
 [ 0.    0.    0.03 -0.53  0.5 ]
 [ 0.    0.    0.    0.04 -0.04]]

Distribuição estacionária pi:
Estado 0 (máquinas operantes): 0.000004
Estado 1 (máquinas operantes): 0.000177
Estado 2 (máquinas operantes): 0.004424
Estado 3 (máquinas operantes): 0.073733
Estado 4 (máquinas operantes): 0.921663

(b) Probabilidade de todas estarem quebradas: 0.00000354
(c) Número médio de máquinas operantes:3.9169


### Questão 6)

In [12]:
import numpy as np

# Parâmetros do modelo
N = 3          # população total
beta = 1.0     # taxa de infecção
gama = 1.0    # taxa de recuperação

# Construir a matriz geradora G (dimensão N+1 = 4)
G = np.zeros((N+1, N+1))

for i in range(N+1):
    # transição i -> i+1 (infecção)
    if i < N:
        lam = beta * i
        G[i, i+1] = lam
    # transição i -> i-1 (recuperação)
    if i > 0:
        mu = gama * i
        G[i, i-1] = mu
    # diagonal = -soma das taxas de saída
    G[i, i] = -G[i].sum()

print("(b) Matriz geradora G:")
print(G)

# Resolver pi * G = 0, com sum(pi) = 1
A = G.T.copy()
A[-1] = np.ones(N+1)   # substitui última linha pela normalização
b = np.zeros(N+1)
b[-1] = 1

pi = np.linalg.solve(A, b)

print("\n(c) Distribuição estacionária pi:")
for i, p in enumerate(pi):
    print(f"pi[{i}] = {p:.6f}")



(b) Matriz geradora G:
[[-0.  0.  0.  0.]
 [ 1. -2.  1.  0.]
 [ 0.  2. -4.  2.]
 [ 0.  0.  3. -3.]]

(c) Distribuição estacionária pi:
pi[0] = 1.000000
pi[1] = -0.000000
pi[2] = -0.000000
pi[3] = 0.000000
